In [14]:
import numpy  as np
from random import uniform
from copy import copy
import math

In [2]:
XMAX = 105
XMIN = -XMAX
YMAX = 195
YMIN = -90

XMID = (XMAX + XMIN) / 2.0
YMID = (YMAX + YMIN) / 2.0

START_X = 0
START_Y = 200

In [3]:
XMID

0.0

In [4]:
def clamp(value, min_value, max_value):
    return min(max(value, min_value), max_value)

In [7]:
class GCODE:
    def __init__(self, name="foobar", feedrate=1000):
        self.start_pos = np.array([START_X, START_Y])
        self.pos = copy(self.start_pos)
        self.f = open(f"{name}.gcode", "wt")
        self.feedrate = feedrate
        
        self.f.write("M17\n")
        self.f.write("M121\n")
        self.f.write("; potatolangelo\n")
        
    def move_to(self, x, y, feedrate=None):
        feedrate = feedrate or self.feedrate
        
        x = clamp(x, XMIN, XMAX)
        y = clamp(y, YMIN, YMAX)
        
        self.pos[0] = x
        self.pos[1] = y
        
        self.f.write(f"G0 X{x} Y{y} F{feedrate}\n")
        
    def step(self, dx, dy, feedrate=None):
        feedrate = feedrate or self.feedrate
        self.pos[0] += dx
        self.pos[1] += dy
        
        x = self.pos[0]
        y = self.pos[1]
        x = clamp(x, XMIN, XMAX)
        y = clamp(y, YMIN, YMAX)
        self.pos[0] = x
        self.pos[1] = y
        
        self.f.write(f"G0 X{x} Y{y} F{feedrate}\n")
        
    def go_to_starting_position(self):
            self.f.write(f"G0 X{self.start_pos[0]} Y{self.start_pos[1]} F{self.feedrate}\n")
            
    def flush(self):
        self.f.flush()

In [8]:
gcode = GCODE("foobar", feedrate=5000)

gcode.move_to(0, 50)
gcode.move_to(-50, 50)
gcode.move_to(-50, 0)
gcode.move_to(-50, -50)
gcode.move_to(0, -50)
gcode.move_to(50, -50)
gcode.move_to(50, 0)
gcode.move_to(50, 50)
gcode.move_to(0, 50)

gcode.go_to_starting_position()

gcode.flush()

In [9]:
gcode = GCODE("random", feedrate=1000)

size = 5

gcode.move_to(XMID, YMID, feedrate=5000)

for _ in range(100):
    gcode.step(
        uniform(-size, size),
        uniform(-size, size)
    )

gcode.go_to_starting_position()

gcode.flush()

In [18]:
size = 5
radius = 0
n_steps = 360 * 2
angle = 0

noise = 1

gcode = GCODE("spiral", feedrate=1500)
gcode.move_to(XMID, YMID, feedrate=5000)


for _ in range(n_steps):
    radius += XMAX / float(n_steps)
    
    gcode.move_to(
        radius * math.sin(angle) + XMID + uniform(-noise, noise),
        radius * math.cos(angle) + YMID + uniform(-noise, noise),
    )
    
    angle += math.radians(1)

gcode.go_to_starting_position()

gcode.flush()